# **Donut 🍩 : Document Understanding Transformer**
Donut 🍩, Document understanding transformer, is a new method of document understanding that utilizes an **OCR-free** end-to-end Transformer model. Donut does not require off-the-shelf OCR engines/APIs, yet it shows state-of-the-art performances on various visual document understanding tasks, such as visual document classification or information extraction (a.k.a. document parsing). In addition, we present SynthDoG 🐶, Synthetic Document Generator, that helps the model pre-training to be flexible on various languages and domains.

### **Setting**

In [1]:
!pip install transformers==4.25.1
!pip install pytorch-lightning==1.6.4
!pip install timm==0.5.4
!pip install gradio
!pip install donut-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.5/585.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.1 which is incompati

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [2]:
import argparse
import gradio as gr
import torch
from PIL import Image

from donut import DonutModel

In [3]:
!unzip /content/DocumentAI_OCR.zip

Archive:  /content/DocumentAI_OCR.zip
   creating: DocumentAI_OCR/
   creating: DocumentAI_OCR/OMG/
  inflating: DocumentAI_OCR/.DS_Store  
  inflating: __MACOSX/DocumentAI_OCR/._.DS_Store  
   creating: DocumentAI_OCR/IVE/
   creating: DocumentAI_OCR/OCR/
   creating: DocumentAI_OCR/Doc/
  inflating: DocumentAI_OCR/OMG/OMG.png  
  inflating: DocumentAI_OCR/OMG/masked1.png  
  inflating: DocumentAI_OCR/OMG/masked0.png  
  inflating: DocumentAI_OCR/OMG/masked2.png  
  inflating: DocumentAI_OCR/OMG/masked3.png  
  inflating: DocumentAI_OCR/IVE/IVE.jpeg  
  inflating: DocumentAI_OCR/IVE/masked1.png  
  inflating: DocumentAI_OCR/IVE/masked0.png  
  inflating: DocumentAI_OCR/IVE/masked2.png  
  inflating: DocumentAI_OCR/OCR/Bumblebee.jpg  
  inflating: __MACOSX/DocumentAI_OCR/OCR/._Bumblebee.jpg  
  inflating: DocumentAI_OCR/OCR/Busanhaeng.jpg  
  inflating: __MACOSX/DocumentAI_OCR/OCR/._Busanhaeng.jpg  
  inflating: DocumentAI_OCR/Doc/.DS_Store  
  inflating: __MACOSX/DocumentAI_OCR/Doc/._

In [4]:
def demo_process_vqa(input_img, question):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pretrained_model.inference(input_img, prompt=user_prompt)["predictions"][0]
    return output


def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

### **Document Classification**

In [5]:
!unzip DocumentAI_OCR.zip

Archive:  DocumentAI_OCR.zip
replace DocumentAI_OCR/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: DocumentAI_OCR/.DS_Store  
replace __MACOSX/DocumentAI_OCR/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace DocumentAI_OCR/OMG/OMG.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="rvlcdip")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-rvlcdip")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:
    task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(args.pretrained_path)

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

pretrained_model.eval()

demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch()

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### **Document VQA**

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="docvqa")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-docvqa")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:
    task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(args.pretrained_path)

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

pretrained_model.eval()

demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# **DiffSTE : Diffusion models for Scene Text Editing**
edit scene text into different font styles and colors following given text instruction. Specifically, we propose to improve pre-trained diffusion models with a dual encoder design, which includes a character encoder for better text legibility and an instruction encoder for better style control. We then utilize an instruction tuning framework to train our model learn the mapping from the text instruction to the corresponding image with either the specified style or the style of the surrounding texts in the background. Such a training method further brings our model the zero-shot generalization ability to the following three scenarios: generating text with unseen font variation, e.g. italic and bold, mixing different fonts to construct a new font, and using more relaxed forms of natural language as the instructions to guide the generation task.



### **Setting**

In [2]:
!unzip /content/DocumentAI_OCR.zip

Archive:  /content/DocumentAI_OCR.zip
replace DocumentAI_OCR/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: DocumentAI_OCR/.DS_Store  
  inflating: __MACOSX/DocumentAI_OCR/._.DS_Store  
  inflating: DocumentAI_OCR/OMG/OMG.png  
  inflating: DocumentAI_OCR/OMG/masked1.png  
  inflating: DocumentAI_OCR/OMG/masked0.png  
  inflating: DocumentAI_OCR/OMG/masked2.png  
  inflating: DocumentAI_OCR/OMG/masked3.png  
  inflating: DocumentAI_OCR/IVE/IVE.jpeg  
  inflating: DocumentAI_OCR/IVE/masked1.png  
  inflating: DocumentAI_OCR/IVE/masked0.png  
  inflating: DocumentAI_OCR/IVE/masked2.png  
  inflating: DocumentAI_OCR/OCR/Bumblebee.jpg  
  inflating: __MACOSX/DocumentAI_OCR/OCR/._Bumblebee.jpg  
  inflating: DocumentAI_OCR/OCR/Busanhaeng.jpg  
  inflating: __MACOSX/DocumentAI_OCR/OCR/._Busanhaeng.jpg  
  inflating: DocumentAI_OCR/Doc/.DS_Store  
  inflating: __MACOSX/DocumentAI_OCR/Doc/._.DS_Store  
  inflating: DocumentAI_OCR/Doc/PRML.png  
  inflating: __MACOSX/DocumentAI

In [3]:
# img resize(256*256) -> https://www.iloveimg.com/ko/resize-image/resize-jpg
# coord -> http://maschek.hu/imagemap/imgmap/
coord = "117,108,233,144"

In [4]:
# character masking
import cv2
import numpy as np

img = '/content/DocumentAI_OCR/OMG/OMG.png'
image = cv2.imread(img)
filename = img.split('/'[-1])

temp = tuple(map(int, coord.split(',')))
x, y, w, h = temp

image2 = np.zeros((image.shape[0], image.shape[1]), dtype="uint8")
cv2.rectangle(image2, (x, y), (w, h), 255, -1)
cv2.imwrite(f'/content/masked_testimg.png', image2)

True

### **Scene Text Editing**

In [5]:
!git clone https://github.com/UCSB-NLP-Chang/DiffSTE.git
%cd DiffSTE

Cloning into 'DiffSTE'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 251 (delta 52), reused 246 (delta 47), pack-reused 0
Receiving objects: 100% (251/251), 6.57 MiB | 22.74 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/DiffSTE


In [ ]:
# requirements.txt -> flax==0.7.2

In [6]:
!curl -sSL https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -o miniconda.sh
!bash ./miniconda.sh -bfp /usr/local
!conda --version
!conda create --name DiffSTE python=3.8 -y

PREFIX=/usr/local
Unpacking payload ...
                                                                                      
Installing base environment...





Preparing transaction: - \ | / - done
Executing transaction: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local
conda 23.5.2
Solving environment: | / done


==> WARNING: A newer version of conda exists. <==
  current version: 23.5.2
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of pa

In [7]:
# pretrained model download
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1fc0RKGWo6MPSJIZNIA_UweTOPai64S9f' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1fc0RKGWo6MPSJIZNIA_UweTOPai64S9f" -O diffste.ckpt && rm -rf ~/cookies.txt

--2023-08-17 06:39:56--  https://docs.google.com/uc?export=download&confirm=t&id=1fc0RKGWo6MPSJIZNIA_UweTOPai64S9f
Resolving docs.google.com (docs.google.com)... 209.85.200.139, 209.85.200.101, 209.85.200.113, ...
Connecting to docs.google.com (docs.google.com)|209.85.200.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-5o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vfsvja3vfamndghar9o83c2o3vfcuaq5/1692254325000/04178247694259295951/*/1fc0RKGWo6MPSJIZNIA_UweTOPai64S9f?e=download&uuid=0737708e-797a-41cc-821a-355e5301f4f5 [following]
--2023-08-17 06:39:56--  https://doc-0o-5o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vfsvja3vfamndghar9o83c2o3vfcuaq5/1692254325000/04178247694259295951/*/1fc0RKGWo6MPSJIZNIA_UweTOPai64S9f?e=download&uuid=0737708e-797a-41cc-821a-355e5301f4f5
Resolving doc-0o-5o-docs.googleusercontent.com (doc-0o-5o-docs.googleusercontent.com)... 173.194.196.132,

In [8]:
# style-free generation
%%bash
source activate DiffSTE
pip install -r requirements.txt
pip install jax --upgrade
pip install triton

python generate.py \
    --ckpt_path /content/DiffSTE/diffste.ckpt \
    --in_image /content/DocumentAI_OCR/OMG/OMG.png \
    --in_mask /content/DocumentAI_OCR/OMG/masked0.png \
    --text Q \
    --out_dir /content/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.4/283.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.6 MB/s eta 0:00:00
  Obtaining dependency information for flax==0.7.2 from https://files.pythonhosted.org/packages/72/a7/147bd0682ff39a4e59352506a7d858e1e003f05a2d96e431fabb8a5491e4/flax-0.7.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 110.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing m

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [9]:
# style-conditional generation
%%bash
source activate DiffSTE
pip install -r requirements.txt
pip install jax --upgrade

python generate.py \
    --ckpt_path /content/DiffSTE/diffste.ckpt \
    --in_image /content/DocumentAI_OCR/OMG/OMG.png \
    --in_mask /content/DocumentAI_OCR/OMG/masked3.png \
    --text QnA \
    --font Caprasimo \
    --color white \
    --out_dir /content/

  Using cached jax-0.4.7-py3-none-any.whl
  Attempting uninstall: jax
    Found existing installation: jax 0.4.13
    Uninstalling jax-0.4.13:
      Successfully uninstalled jax-0.4.13
  Using cached jax-0.4.13-py3-none-any.whl
  Attempting uninstall: jax
    Found existing installation: jax 0.4.7
    Uninstalling jax-0.4.7:
      Successfully uninstalled jax-0.4.7
Initialize vae from finetuned...
Initialize CharTokenizer
Initilize char embedder...
AutoencoderKL has 83.65 M params.
UNet2DMultiConditionModel has 1108.40 M params.
CLIPTextModel has 123.06 M params.
CharEmbedder has 0.00 M params.


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


# **EasyOCR**
Ready-to-use OCR with 80+ supported languages and all popular writing scripts including: Latin, Chinese, Arabic, Devanagari, Cyrillic, etc.
- opensource text detection & recognition model
- support language -> https://www.jaided.ai/easyocr/

### **Setting**

In [1]:
!pip install easyocr

In [2]:
!unzip DocumentAI.zip

unzip:  cannot find or open DocumentAI.zip, DocumentAI.zip.zip or DocumentAI.zip.ZIP.


### **Detection & Crop**

In [12]:
def crop_box(ocr):
    max_diff = 0
    max_row = []
    for i in ocr:
        diff = i[0][2][1]-i[0][0][1]
        x_diff = i[0][2][0]-i[0][0][0]
        if diff > max_diff and x_diff > 0:
            max_diff = diff
            max_row = i[0]
    box = tuple(max_row[0] + max_row[2])
    return box

In [13]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def img_show(img):
    img = np.array(img)
    plt.imshow(img)

### **English OCR**

In [14]:
import easyocr

en_imgpath = "/content/DocumentAI_OCR/OCR/Bumblebee.jpg"
en_img = Image.open(en_imgpath)
img_show(en_img)

ModuleNotFoundError: ignored

In [ ]:
en_reader = easyocr.Reader(['en'])
en_ocr = en_reader.readtext(en_imgpath)
box = crop_box(en_ocr)
text = en_img.crop(box)
img_show(text)

In [ ]:
# recognition
print(en_ocr)

### **Korean OCR**

In [ ]:
ko_imgpath = "/content/DocumentAI_OCR/OCR/Busanhaeng.jpg"
ko_img = Image.open(ko_imgpath)
img_show(ko_img)

In [ ]:
ko_reader = easyocr.Reader(['ko'])
ko_ocr = ko_reader.readtext(ko_imgpath)
box = crop_box(ko_ocr)
text = ko_img.crop(box)
img_show(text)

In [ ]:
# recognition
print(ko_ocr)